In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.nn.functional import cross_entropy
import numpy as np
import re, string
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter
import json
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zynicide/wine-reviews" )
# Parameters
VOCAB_SIZE = 10000
MAX_LEN = 80
EMBEDDING_DIM = 256
N_HEADS = 2
FF_DIM = 256
BATCH_SIZE = 32
EPOCHS = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Load the data
with open(path+"/winemag-data-130k-v2.json") as json_data:
    wine_data = json.load(json_data)
filtered_data = [
    "wine review : "
    + x["country"]
    + " : "
    + x["province"]
    + " : "
    + x["variety"]
    + " : "
    + x["description"]
    for x in wine_data
    if x["country"] is not None
    and x["province"] is not None
    and x["variety"] is not None
    and x["description"] is not None
]
# Tokenizer
class SimpleTokenizer:
    def __init__(self, texts, vocab_size):
        self.vocab_size = vocab_size
        self.counter = Counter()
        for text in texts:
            self.counter.update(text.split())
        self.vocab = [word for word, _ in self.counter.most_common(vocab_size - 2)]
        self.word2idx = {word: idx + 2 for idx, word in enumerate(self.vocab)}
        self.word2idx["<pad>"] = 0
        self.word2idx["<unk>"] = 1

    def encode(self, text):
        return [self.word2idx.get(word, 1) for word in text.split()]

# Data preparation
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s.lower()


# Create tokenizer
tokenizer = SimpleTokenizer(filtered_data, VOCAB_SIZE)


In [2]:
# Dataset class
class WineDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.tokenizer.encode(self.texts[idx])[:self.max_len+1]
        padding = [self.tokenizer.word2idx["<pad>"]] * (self.max_len + 1 - len(tokens))
        tokens += padding
        return torch.tensor(tokens[:-1]), torch.tensor(tokens[1:])

train_dataset = WineDataset([pad_punctuation(t) for t in filtered_data], tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


In [3]:
class GPTBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.ln1 = nn.LayerNorm(embed_dim)
        self.ln2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        seq_len = x.size(1)
        mask = torch.tril(torch.ones(seq_len, seq_len)).to(x.device)
        mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, 0.0)

        attn_output, _ = self.attn(x, x, x, attn_mask=mask)
        x = self.ln1(x + attn_output)
        ffn_output = self.ffn(x)
        return self.ln2(x + ffn_output)


In [4]:
class GPTModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, max_len, num_heads, ff_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(max_len, embed_dim)
        self.transformer = GPTBlock(embed_dim, num_heads, ff_dim)
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        positions = torch.arange(0, x.size(1), device=x.device).unsqueeze(0)
        x = self.embedding(x) + self.pos_embedding(positions)
        x = self.transformer(x)
        return self.fc(x)


In [5]:
# Training loop
# Training
model = GPTModel(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, N_HEADS, FF_DIM).to(DEVICE)
optimizer = Adam(model.parameters(), lr=0.0001)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for x_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
        optimizer.zero_grad()
        logits = model(x_batch)
        loss = cross_entropy(logits.view(-1, VOCAB_SIZE), y_batch.view(-1), ignore_index=tokenizer.word2idx['<pad>'])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_loss:.4f}")



Epoch 1/100: 100%|██████████| 4060/4060 [00:54<00:00, 74.48it/s]


Epoch 1, Training Loss: 4.0331


Epoch 2/100: 100%|██████████| 4060/4060 [00:53<00:00, 75.39it/s]


Epoch 2, Training Loss: 3.4174


Epoch 3/100: 100%|██████████| 4060/4060 [00:53<00:00, 75.53it/s]


...

Epoch 98, Training Loss: 2.3311


Epoch 99/100: 100%|██████████| 4060/4060 [00:53<00:00, 75.88it/s]


Epoch 99, Training Loss: 2.3297


Epoch 100/100: 100%|██████████| 4060/4060 [00:53<00:00, 75.64it/s]

Epoch 100, Training Loss: 2.3284


In [6]:
def generate_text(model, tokenizer, start_prompt, max_tokens=80, temperature=1.0):
    model.eval()
    tokens = tokenizer.encode(start_prompt)
    tokens = tokens[:MAX_LEN]
    generated = tokens.copy()

    with torch.no_grad():
        for _ in range(max_tokens):
            input_tensor = torch.tensor([generated[-MAX_LEN:]], device=DEVICE)
            logits = model(input_tensor)
            logits = logits[:, -1, :] / temperature
            probs = torch.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()

            if next_token == tokenizer.word2idx['<pad>']:
                break

            generated.append(next_token)

    idx2word = {idx: word for word, idx in tokenizer.word2idx.items()}
    generated_text = ' '.join(idx2word.get(idx, '<unk>') for idx in generated)
    return generated_text

# Example usage:
prompt = "wine review : US"
generated_text = generate_text(model, tokenizer, prompt, max_tokens=50, temperature=0.8).strip()
print("Generated text:\n", generated_text)


Generated text:
 wine review : US : <unk> : <unk> : a <unk> : with a ripe , slight green apple character that gives the wine . the texture of the wine , perfumed , white fruits are lifted by the apricot fruits . the wine is bright , ready to drink . wait until 2017
